### Fairness anaysis 

In [2]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import numpy as np

# Set device to CUDA if available, otherwise fall back to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [ ]:
# Load your original dataset
df = pd.read_csv("/home/hailemicaelyimer/fairness/Data_working_now/cleaned_metadata.csv")

print("=== DATASET BASELINE FAIRNESS ANALYSIS ===")
print(f"Total samples: {len(df)}")

# Basic demographics
print(f"\nCOVID Distribution:")
covid_dist = df['y_true'].value_counts(normalize=True)
print(f"  COVID Positive: {covid_dist[1]:.3f} ({covid_dist[1]*len(df):.0f} cases)")
print(f"  COVID Negative: {covid_dist[0]:.3f} ({covid_dist[0]*len(df):.0f} cases)")

print(f"\nGender Distribution:")
gender_dist = df['Gender_encoded'].value_counts(normalize=True)
for gender, prop in gender_dist.items():
    print(f"  Gender {gender}: {prop:.3f} ({prop*len(df):.0f} cases)")

print(f"\nAge Distribution:")
print(f"  Mean age (normalized): {df['Age_normalized'].mean():.3f}")
print(f"  Std age (normalized): {df['Age_normalized'].std():.3f}")

print(f"\nCountry Distribution:")
country_dist = df['Country_encoded'].value_counts(normalize=True)
for country, prop in country_dist.items():
    print(f"  Country {country}: {prop:.3f} ({prop*len(df):.0f} cases)")

print(f"\nInstitution Distribution:")
inst_dist = df['Institution_encoded'].value_counts(normalize=True)
for inst, prop in inst_dist.items():
    print(f"  Institution {inst}: {prop:.3f} ({prop*len(df):.0f} cases)")

# Now analyze inherent bias in the dataset (ground truth labels)
print("\n" + "="*60)
print("INHERENT DATASET BIAS ANALYSIS")
print("="*60)

def analyze_group_bias(df, group_col, label_col='y_true'):
    """Analyze bias in ground truth labels by demographic groups"""
    group_stats = []
    
    for group_val in sorted(df[group_col].unique()):
        group_df = df[df[group_col] == group_val]
        covid_rate = group_df[label_col].mean()
        group_size = len(group_df)
        
        group_stats.append({
            'Group': f"{group_col}={group_val}",
            'Size': group_size,
            'COVID_Rate': covid_rate,
            'Percentage': group_size / len(df) * 100
        })
    
    return pd.DataFrame(group_stats)

# Analyze inherent bias in ground truth labels
print("\n1. AGE GROUP BIAS IN GROUND TRUTH:")
# Create age bins for analysis
df['age_binned'] = pd.qcut(df['Age_normalized'], q=3, labels=['Young', 'Middle', 'Older'], duplicates='drop')
age_bias = analyze_group_bias(df, 'age_binned')
print(age_bias.to_string(index=False))

age_covid_rates = age_bias['COVID_Rate'].values
age_dpd_ground_truth = age_covid_rates.max() - age_covid_rates.min()
print(f"Ground Truth Age DPD: {age_dpd_ground_truth:.4f}")

print("\n2. GENDER GROUP BIAS IN GROUND TRUTH:")
gender_bias = analyze_group_bias(df, 'Gender_encoded')
print(gender_bias.to_string(index=False))

gender_covid_rates = gender_bias['COVID_Rate'].values
gender_dpd_ground_truth = gender_covid_rates.max() - gender_covid_rates.min()
print(f"Ground Truth Gender DPD: {gender_dpd_ground_truth:.4f}")

print("\n3. COUNTRY GROUP BIAS IN GROUND TRUTH:")
country_bias = analyze_group_bias(df, 'Country_encoded')
print(country_bias.to_string(index=False))

country_covid_rates = country_bias['COVID_Rate'].values
country_dpd_ground_truth = country_covid_rates.max() - country_covid_rates.min()
print(f"Ground Truth Country DPD: {country_dpd_ground_truth:.4f}")

print("\n4. INSTITUTION GROUP BIAS IN GROUND TRUTH:")
inst_bias = analyze_group_bias(df, 'Institution_encoded')
print(inst_bias.to_string(index=False))

inst_covid_rates = inst_bias['COVID_Rate'].values
inst_dpd_ground_truth = inst_covid_rates.max() - inst_covid_rates.min()
print(f"Ground Truth Institution DPD: {inst_dpd_ground_truth:.4f}")



=== DATASET BASELINE FAIRNESS ANALYSIS ===
Total samples: 12611

COVID Distribution:
  COVID Positive: 0.453 (5718 cases)
  COVID Negative: 0.547 (6893 cases)

Gender Distribution:
  Gender 1: 0.620 (7818 cases)
  Gender 0: 0.380 (4793 cases)

Age Distribution:
  Mean age (normalized): 0.000
  Std age (normalized): 1.000

Country Distribution:
  Country 1: 0.515 (6493 cases)
  Country 3: 0.465 (5865 cases)
  Country 2: 0.008 (103 cases)
  Country 0: 0.006 (72 cases)
  Country 4: 0.003 (44 cases)
  Country 5: 0.003 (34 cases)

Institution Distribution:
  Institution 2: 0.465 (5865 cases)
  Institution 0: 0.363 (4575 cases)
  Institution 3: 0.136 (1719 cases)
  Institution 7: 0.016 (199 cases)
  Institution 4: 0.008 (103 cases)
  Institution 5: 0.006 (72 cases)
  Institution 6: 0.003 (44 cases)
  Institution 1: 0.003 (34 cases)

INHERENT DATASET BIAS ANALYSIS

1. AGE GROUP BIAS IN GROUND TRUTH:
            Group  Size  COVID_Rate  Percentage
age_binned=Middle  3986    0.534621   31.60732

#### Fairness mitigation with causal model

In [ ]:
# Define paths
base_dir = "/home/hailemicaelyimer/fairness/Data_working_now/curated_data/curated_data"
covid_dir = os.path.join(base_dir, "2COVID")
non_covid_dir = os.path.join(base_dir, "1NonCOVID")
csv_path = "/home/hailemicaelyimer/fairness/Data_working_now/cleaned_metadata.csv"
output_dir = "/home/hailemicaelyimer/fairness/resnettt50"
os.makedirs(output_dir, exist_ok=True)

# Load CSV
df = pd.read_csv(csv_path)
print(f"Loaded CSV with {len(df)} entries")

# Concept columns (including task as last)
concept_cols = ['Age_normalized', 'Gender_encoded', 'Country_encoded', 'Institution_encoded', 'y_true']

# Remove duplicates based on patient/image context (e.g., same File name)
df = df.drop_duplicates(subset=['File name'])
print(f"After removing duplicates: {len(df)} entries")

# Normalize categorical encoded to 0-1 using max values from the dataset
max_country = df['Country_encoded'].max()
max_institution = df['Institution_encoded'].max()
df['Country_encoded'] /= max_country
df['Institution_encoded'] /= max_institution

# Clean data: Drop NaNs and clip to [0,1] to prevent loss errors
df = df.dropna(subset=concept_cols)
df[concept_cols] = df[concept_cols].clip(0, 1)  # Clip all to [0,1], including Age_normalized
print(f"After cleaning: {len(df)} entries")

# Custom Dataset
class C2BMDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.concepts = torch.tensor(df[concept_cols].values, dtype=torch.float32)  # All concepts + task
        self.filenames = df['File name'].tolist()
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.filenames[idx]
        base_name = os.path.splitext(file_name)[0]
        covid_path = os.path.join(covid_dir, f"{base_name}.png")
        non_covid_path = os.path.join(non_covid_dir, f"{base_name}.png")
        
        # Check COVID directory first
        if os.path.exists(covid_dir) and os.path.isfile(covid_path):
            img_path = covid_path
        # Then check Non-COVID directory
        elif os.path.exists(non_covid_dir) and os.path.isfile(non_covid_path):
            img_path = non_covid_path
        else:
            raise FileNotFoundError(f"Image not found for {file_name} in either COVID or Non-COVID directories")
        
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        concepts = self.concepts[idx]
        return image, concepts, img_path

# Transformations
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Split data
train_df, temp_df = train_test_split(df, test_size=0.30, random_state=42, stratify=df['y_true'])
val_df, test_df = train_test_split(temp_df, test_size=0.50, random_state=42, stratify=temp_df['y_true'])

# Datasets
train_dataset = C2BMDataset(train_df, transform=transform)
val_dataset = C2BMDataset(val_df, transform=transform)
test_dataset = C2BMDataset(test_df, transform=transform)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# C2BM Model with Dropout for Regularization using ResNet-50
class C2BMModel(nn.Module):
    def __init__(self):
        super(C2BMModel, self).__init__()
        # Use ResNet-50 instead of EfficientNet-B0
        self.image_encoder = models.resnet50(weights=None)  # Use ResNet-50, no pretrained weights
        self.image_encoder.fc = nn.Identity()  # Remove final fully connected layer to get 2048 features
        
        self.num_concepts = 5  # Age(0), Gender(1), Country(2), Institution(3), COVID(4)
        self.u_dim = 64
        self.u_encoder = nn.Linear(2048, self.num_concepts * self.u_dim)  # Adjusted for ResNet-50 output (2048 features)
        
        # Causal graph parents
        self.parents = {
            0: [],  # Age
            1: [],  # Gender
            2: [],  # Country
            3: [2],  # Institution <- Country
            4: [0, 1, 3]  # COVID <- Age, Gender, Institution
        }
        
        # Meta-models for each concept with dropout
        self.meta_models = nn.ModuleList()
        for i in range(self.num_concepts):
            num_pa = len(self.parents[i])
            if num_pa == 0:  # Sources: MLP to logit
                self.meta_models.append(nn.Sequential(
                    nn.Linear(self.u_dim, 32),
                    nn.ReLU(),
                    nn.Dropout(0.5),  # Add dropout for regularization
                    nn.Linear(32, 1)
                ))
            else:  # Non-sources: MLP to weights (theta) for parents
                self.meta_models.append(nn.Sequential(
                    nn.Linear(self.u_dim, 32),
                    nn.ReLU(),
                    nn.Dropout(0.5),  # Add dropout for regularization
                    nn.Linear(32, num_pa)
                ))
        # Initialize weights
        for module in self.meta_models:
            for layer in module:
                if isinstance(layer, nn.Linear):
                    nn.init.xavier_uniform_(layer.weight)
                    nn.init.constant_(layer.bias, 0.0)

    def forward(self, image, intervention=None):
        img_feat = self.image_encoder(image)
        u_flat = self.u_encoder(img_feat)
        U = u_flat.view(-1, self.num_concepts, self.u_dim)
        
        V = [None] * self.num_concepts
        for i in range(self.num_concepts):  # Topological order
            u_i = U[:, i, :]
            pa = self.parents[i]
            if len(pa) == 0:
                logit = self.meta_models[i](u_i)
            else:
                theta = self.meta_models[i](u_i)
                pa_v = torch.stack([V[j] for j in pa], dim=1)
                logit = torch.sum(theta.unsqueeze(-1) * pa_v, dim=1)
            
            v_i = torch.sigmoid(logit)
            
            if intervention and i in intervention:
                v_i = torch.full_like(v_i, intervention[i])
            
            V[i] = v_i
        
        return torch.cat(V, dim=1)  # Return all predicted concepts (last is COVID prob)

# Instantiate model, optimizer with try-except for CUDA issues
try:
    model = C2BMModel().to(device)
except RuntimeError as e:
    print(f"CUDA Error: {e}. Falling back to CPU.")
    device = torch.device("cpu")
    model = C2BMModel().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training epoch with mixed loss
def train_epoch(model, loader, optimizer, device):
    model.train()
    running_loss = 0.0
    bce = nn.BCELoss()
    mse = nn.MSELoss()
    binary_indices = [1, 4]  # Gender, COVID
    cont_indices = [0, 2, 3]  # Age, Country, Institution
    for images, concepts, _ in loader:
        images, concepts = images.to(device), concepts.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = 0
        for i in binary_indices:
            loss += bce(outputs[:, i], concepts[:, i])
        for i in cont_indices:
            loss += mse(outputs[:, i], concepts[:, i])
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(loader)

# Evaluation (with optional intervention for fairness) with mixed loss
def evaluate(model, loader, device, intervention=None):
    model.eval()
    preds = []
    true_labels = []
    running_loss = 0.0
    bce = nn.BCELoss()
    mse = nn.MSELoss()
    binary_indices = [1, 4]  # Gender, COVID
    cont_indices = [0, 2, 3]  # Age, Country, Institution
    with torch.no_grad():
        for images, concepts, _ in loader:
            images, concepts = images.to(device), concepts.to(device)
            outputs = model(images, intervention=intervention)
            loss = 0
            for i in binary_indices:
                loss += bce(outputs[:, i], concepts[:, i])
            for i in cont_indices:
                loss += mse(outputs[:, i], concepts[:, i])
            running_loss += loss.item()
            predicted = (outputs[:, -1] > 0.5).float().cpu().numpy()  # COVID pred
            preds.extend(predicted)
            true_labels.extend(concepts[:, -1].cpu().numpy())
    acc = accuracy_score(true_labels, preds)
    prec = precision_score(true_labels, preds)
    rec = recall_score(true_labels, preds)
    f1 = f1_score(true_labels, preds)
    return acc, prec, rec, f1, running_loss / len(loader)

# Training loop with early stopping
num_epochs = 50  # Increased from 5 to allow more training with early stopping
early_stopping_patience = 9
best_val_f1 = 0.0
patience_counter = 0
best_model_path = "/home/hailemicaelyimer/fairness/resnettt50/c2bm_model_best.pth"
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    train_loss = train_epoch(model, train_loader, optimizer, device)
    val_acc, val_prec, val_rec, val_f1, val_loss = evaluate(model, val_loader, device)
    
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    
    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}, Precision: {val_prec:.4f}, Recall: {val_rec:.4f}, F1: {val_f1:.4f}")
    
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        patience_counter = 0
        torch.save(model.state_dict(), best_model_path)
        print(f"Saved best model with Val F1: {best_val_f1:.4f}")
    else:
        patience_counter += 1
        if patience_counter >= early_stopping_patience:
            print(f"Early stopping triggered after {epoch+1} epochs")
            break

# Plot losses
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(train_losses) + 1), train_losses, label='Training Loss')
plt.plot(range(1, len(val_losses) + 1), val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Over Time')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(output_dir, 'c2bm_loss_plot.png'))
plt.close()
print(f"Loss plot saved to {os.path.join(output_dir, 'c2bm_loss_plot.png')}")

# Load best model and evaluate on test set (with example intervention for fairness)
model.load_state_dict(torch.load(best_model_path, weights_only=True))
intervention = {3: 0}  # Set Institution to 0.5 to block Country effect
test_acc, test_prec, test_rec, test_f1, test_loss = evaluate(model, test_loader, device, intervention=intervention)
print(f"\nTest Results (with fairness intervention on Institution):")
print(f"Test Loss: {test_loss:.4f}, Accuracy: {test_acc:.4f}, Precision: {test_prec:.4f}, Recall: {test_rec:.4f}, F1: {test_f1:.4f}")

# Save final model
torch.save(model.state_dict(), "/home/hailemicaelyimer/fairness/faurness/c2bm_model_final.pth")
print("C2BM model saved successfully")

Using device: cuda
Loaded CSV with 12611 entries
After removing duplicates: 12611 entries
After cleaning: 12611 entries
Epoch 1/50
Train Loss: 1.7709
Val Loss: 1.3453, Accuracy: 0.8103, Precision: 0.7570, Recall: 0.8566, F1: 0.8037
Saved best model with Val F1: 0.8037
Epoch 2/50
Train Loss: 1.3831
Val Loss: 2.5485, Accuracy: 0.7463, Precision: 0.9163, Recall: 0.4848, F1: 0.6341
Epoch 3/50
Train Loss: 1.3889
Val Loss: 1.9500, Accuracy: 0.5877, Precision: 0.7868, Recall: 0.1247, F1: 0.2153
Epoch 4/50
Train Loss: 1.3471
Val Loss: 1.4856, Accuracy: 0.8134, Precision: 0.7595, Recall: 0.8613, F1: 0.8072
Saved best model with Val F1: 0.8072
Epoch 5/50
Train Loss: 1.2844
Val Loss: 1.4340, Accuracy: 0.7146, Precision: 1.0000, Recall: 0.3706, F1: 0.5408
Epoch 6/50
Train Loss: 1.2369
Val Loss: 1.8278, Accuracy: 0.8927, Precision: 0.9304, Recall: 0.8252, F1: 0.8746
Saved best model with Val F1: 0.8746
Epoch 7/50
Train Loss: 1.2336
Val Loss: 1.1829, Accuracy: 0.8895, Precision: 0.9577, Recall: 0.79

#### Analysis of the causal model

In [20]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from scipy.stats import chi2_contingency

# Modified Evaluation Function to collect per-sample data
def evaluate(model, loader, device, intervention=None):
    model.eval()
    all_preds = []
    all_true = []
    all_concepts = []
    running_loss = 0.0
    bce = nn.BCELoss()
    mse = nn.MSELoss()
    binary_indices = [1, 4]  # Gender, COVID
    cont_indices = [0, 2, 3]  # Age, Country, Institution
    with torch.no_grad():
        for images, concepts, _ in loader:
            images, concepts = images.to(device), concepts.to(device)
            outputs = model(images, intervention=intervention)
            loss = 0
            for i in binary_indices:
                loss += bce(outputs[:, i], concepts[:, i])
            for i in cont_indices:
                loss += mse(outputs[:, i], concepts[:, i])
            running_loss += loss.item()
            
            predicted = (outputs[:, -1] > 0.5).float().cpu().numpy()
            true_labels = concepts[:, -1].cpu().numpy()
            concepts_np = concepts.cpu().numpy()
            all_preds.extend(predicted)
            all_true.extend(true_labels)
            all_concepts.extend(concepts_np)
    
    acc = accuracy_score(all_true, all_preds)
    prec = precision_score(all_true, all_preds, zero_division=0)
    rec = recall_score(all_true, all_preds, zero_division=0)
    f1 = f1_score(all_true, all_preds, zero_division=0)
    
    results_df = pd.DataFrame(all_concepts, columns=concept_cols)
    results_df['y_pred'] = all_preds
    results_df['y_true'] = all_true
    
    return acc, prec, rec, f1, running_loss / len(loader), results_df

# Function to compute metrics for a subgroup
def compute_metrics(group_df):
    y_true = group_df['y_true']
    y_pred = group_df['y_pred']
    if len(y_true) < 10 or len(set(y_true)) < 2:  # Require min 10 samples
        return None
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    pr = y_pred.mean()
    tpr = rec
    fpr = ((y_pred == 1) & (y_true == 0)).sum() / (y_true == 0).sum() if (y_true == 0).sum() > 0 else 0
    fdr = 1 - prec if prec > 0 else 0
    return {
        'Size': len(group_df),
        'Accuracy': acc,
        'Precision': prec,
        'Recall': tpr,
        'F1': f1,
        'Positive Rate': pr,
        'TPR': tpr,
        'FPR': fpr,
        'FDR': fdr
    }

# Evaluate with and without intervention
model.load_state_dict(torch.load(best_model_path, weights_only=True))
intervention = {3: 0}
test_acc, test_prec, test_rec, test_f1, test_loss, test_df_int = evaluate(model, test_loader, device, intervention=intervention)
test_acc_no_int, _, _, _, _, test_df_no_int = evaluate(model, test_loader, device, intervention=None)

# Bin continuous attributes directly on test_df_int and test_df_no_int
for df in [test_df_int, test_df_no_int]:
    for attr in ['Age_normalized', 'Country_encoded', 'Institution_encoded']:
        try:
            if len(df[attr].unique()) > 4:  # Bin only if enough unique values
                df[f'{attr}_binned'] = pd.qcut(df[attr], q=4, duplicates='drop', labels=False)
                # Map to meaningful labels
                if attr == 'Age_normalized':
                    df[f'{attr}_binned'] = df[f'{attr}_binned'].map({
                        0: 'Young', 1: 'Middle-aged', 2: 'Older', 3: 'Elderly'
                    })
                else:
                    df[f'{attr}_binned'] = df[f'{attr}_binned'].map({
                        0: 'Group 0', 1: 'Group 1', 2: 'Group 2', 3: 'Group 3'
                    })
            else:
                df[f'{attr}_binned'] = df[attr]  # Use original values if too few
        except ValueError as e:
            print(f"Warning: Could not bin {attr}: {e}. Using original values.")
            df[f'{attr}_binned'] = df[attr]

print(f"\nTest Results (with fairness intervention on Institution):")
print(f"Test Loss: {test_loss:.4f}, Accuracy: {test_acc:.4f}, Precision: {test_prec:.4f}, Recall: {test_rec:.4f}, F1: {test_f1:.4f}")
print(f"Test Accuracy (no intervention): {test_acc_no_int:.4f}")

# Group fairness analysis
group_attrs = ['Age_normalized_binned', 'Gender_encoded', 'Country_encoded_binned', 'Institution_encoded_binned']
fairness_results = []

for attr in group_attrs:
    unique_vals = sorted(test_df_int[attr].unique(), key=lambda x: str(x))
    subgroup_metrics_int = {}
    subgroup_metrics_no_int = {}
    pr_list_int = []
    tpr_list_int = []
    fpr_list_int = []
    fdr_list_int = []
    pr_list_no_int = []
    valid_vals = []
    
    for val in unique_vals:
        group_int = test_df_int[test_df_int[attr] == val]
        group_no_int = test_df_no_int[test_df_no_int[attr] == val]
        metrics_int = compute_metrics(group_int)
        metrics_no_int = compute_metrics(group_no_int)
        if metrics_int:
            subgroup_metrics_int[val] = metrics_int
            pr_list_int.append(metrics_int['Positive Rate'])
            tpr_list_int.append(metrics_int['TPR'])
            fpr_list_int.append(metrics_int['FPR'])
            fdr_list_int.append(metrics_int['FDR'])
            subgroup_metrics_no_int[val] = metrics_no_int or {}
            pr_list_no_int.append(metrics_no_int['Positive Rate'] if metrics_no_int else 0)
            valid_vals.append(val)
    
    if len(pr_list_int) < 2:
        print(f"\nSkipping {attr}: Not enough subgroups for disparity calculation.")
        continue
    
    # Fairness metrics
    dpd_int = max(pr_list_int) - min(pr_list_int)
    di_int = min(pr_list_int) / max(pr_list_int) if max(pr_list_int) > 0 else 0.0
    tpr_diff_int = max(tpr_list_int) - min(tpr_list_int)
    fpr_diff_int = max(fpr_list_int) - min(fpr_list_int)
    eod_int = max(tpr_diff_int, fpr_diff_int)
    fdrd_int = max(fdr_list_int) - min(fdr_list_int)
    
    dpd_no_int = max(pr_list_no_int) - min(pr_list_no_int) if pr_list_no_int else 0.0
    di_no_int = min(pr_list_no_int) / max(pr_list_no_int) if max(pr_list_no_int) > 0 else 0.0
    
    # Chi-squared test
    contingency = pd.crosstab(test_df_int[attr], test_df_int['y_pred'])
    chi2, p_value, _, _ = chi2_contingency(contingency) if contingency.shape[1] > 1 else (0, 1.0)
    
    print(f"\nGroup Fairness for {attr} (with intervention):")
    print(f"Demographic Parity Difference: {dpd_int:.6f} (No intervention: {dpd_no_int:.6f})")
    print(f"Disparate Impact: {di_int:.6f} (No intervention: {di_no_int:.6f})")
    print(f"Equalized Odds Difference: {eod_int:.6f}")
    print(f"False Discovery Rate Difference: {fdrd_int:.6f}")
    print(f"Chi-squared p-value: {p_value:.6f}")
    
    print(f"\nPer-Subgroup Metrics for {attr} (with intervention):")
    for val in valid_vals:
        metrics = subgroup_metrics_int[val]
        print(f"  Value {val}:")
        print(f"    Size: {metrics['Size']}")
        print(f"    Positive Rate: {metrics['Positive Rate']:.6f}")
        print(f"    TPR: {metrics['TPR']:.6f}")
        print(f"    FPR: {metrics['FPR']:.6f}")
        print(f"    FDR: {metrics['FDR']:.6f}")
    
    # Visualization
    plt.figure(figsize=(10, 6))
    sns.boxplot(x=attr, y='y_pred', hue=test_df_int['y_pred'].map({0: 'Negative', 1: 'Positive'}), data=test_df_int)
    plt.title(f'Positive Rate by {attr}')
    plt.savefig(os.path.join(output_dir, f'positive_rate_by_{attr}.png'))
    plt.close()
    
    fairness_results.append({
        'Group': attr,
        'Demographic Parity Difference (Int)': dpd_int,
        'Disparate Impact (Int)': di_int,
        'Equalized Odds Difference (Int)': eod_int,
        'False Discovery Rate Difference (Int)': fdrd_int,
        'DPD (No Int)': dpd_no_int,
        'DI (No Int)': di_no_int,
        'Chi-squared p-value': p_value,
        **{f'Positive Rate ({val}, Int)': subgroup_metrics_int[val]['Positive Rate'] for val in valid_vals},
        **{f'TPR ({val}, Int)': subgroup_metrics_int[val]['TPR'] for val in valid_vals},
        **{f'FPR ({val}, Int)': subgroup_metrics_int[val]['FPR'] for val in valid_vals},
        **{f'FDR ({val}, Int)': subgroup_metrics_int[val]['FDR'] for val in valid_vals}
    })

# Save results
fairness_df = pd.DataFrame(fairness_results)
fairness_df.to_csv(os.path.join(output_dir, '/home/hailemicaelyimer/fairness/resnettt50/c2bm_fairness_metrics_enhanced.csv'), index=False)
print(f"\nEnhanced fairness metrics saved to {os.path.join(output_dir, 'c2bm_fairness_metrics_enhanced.csv')}")


Test Results (with fairness intervention on Institution):
Test Loss: 0.5989, Accuracy: 0.9836, Precision: 0.9803, Recall: 0.9837, F1: 0.9820
Test Accuracy (no intervention): 0.9841

Group Fairness for Age_normalized_binned (with intervention):
Demographic Parity Difference: 0.057246 (No intervention: 0.060976)
Disparate Impact: 0.877876 (No intervention: 0.869725)
Equalized Odds Difference: 0.013853
False Discovery Rate Difference: 0.004612
Chi-squared p-value: 0.037706

Per-Subgroup Metrics for Age_normalized_binned (with intervention):
  Value Middle-aged:
    Size: 452
    Positive Rate: 0.411504
    TPR: 0.994565
    FPR: 0.011194
    FDR: 0.016129
  Value Young:
    Size: 1440
    Positive Rate: 0.468750
    TPR: 0.980712
    FPR: 0.018277
    FDR: 0.020741

Group Fairness for Gender_encoded (with intervention):
Demographic Parity Difference: 0.018478 (No intervention: 0.016484)
Disparate Impact: 0.960404 (No intervention: 0.964462)
Equalized Odds Difference: 0.019773
False Disco